# The task is to process two tables.

1. In "feed_actions" for each user, we count the number of views and likes of the content.  
   In "message_actions" for each user, we count how many messages they receive and send, how many people they write to, how many people write to them.
2. Then we combine two tables into one.
   For this table, we calculate all these metrics in the context of "gender", "age" and "os".
3. Then we write the total data with all the metrics into a total dataframe.

The structure of the final table should be like this:  
Date - "event_date"  
Slice name - "dimension"  
Slice value - "dimension_value"  
Number of views - "views"  
Number of likes - "likes"  
Number of messages received - "messages_received"  
Number of messages sent - "messages_sent"  
Number of users received messages - "users_received"  
Number of users sent a message - "users_sent"

In [1]:
# Download the requered libraries
import pandas as pd
import pandahouse

from datetime import datetime, timedelta

In [2]:
# Form a query from feed_actions database

query_act = """
        SELECT 
          user_id
        , countIf(action = 'like') as likes        
        , countIf(action = 'view') as views
        , CASE gender
            WHEN 0 THEN 'female'
            ELSE 'male'
          END AS gender
        , CASE 
            WHEN age > 65 THEN 'old'
            WHEN age > 19 AND age <= 65 THEN 'adult'
            ELSE 'young'
          END AS age
        , os        
        FROM simulator_20230120.feed_actions
        WHERE toDate(time) = today() - 1
        GROUP BY user_id, gender, age, os    
        """

In [3]:
# Form a query from message_actions database
query_mess = """
            SELECT
                  user_id
                , l.messages_sent
                , r.messages_received
                , l.users_sent
                , r.users_received 
                FROM                    
                    (SELECT 
                      user_id 
                    , count(distinct reciever_id) AS users_sent
                    , count() AS messages_sent
                    
                    FROM simulator_20230120.message_actions
                    WHERE toDate(time) = today() - 1
                    GROUP BY user_id) AS l

                    LEFT JOIN

                    (SELECT 
                      reciever_id 
                    , count(distinct user_id) AS users_received
                    , count() AS messages_received                    
                    FROM simulator_20230120.message_actions
                    WHERE toDate(time) = today() - 1
                    GROUP BY reciever_id) AS r
                    
                    ON l.user_id = r.reciever_id
            """

In [4]:
# Create connection

connection = {'host': 'https://clickhouse.lab.karpov.courses',
              'password': 'dpo_python_2020',
              'user': 'student',
              'database': 'simulator'
                }

In [5]:
# Get information from databases and form two dataframes

df_feed = pandahouse.read_clickhouse(query=query_act, connection=connection)
df_mess = pandahouse.read_clickhouse(query=query_mess, connection=connection)

In [6]:
df_feed.head()

,user_id,likes,views,gender,age,os
0,60005,9,20,female,young,Android
1,87792,6,33,male,young,Android
2,65784,8,32,male,young,Android
3,167330,4,34,male,adult,Android
4,3317,10,62,male,adult,Android


In [7]:
df_mess.head()

,user_id,messages_sent,messages_received,users_sent,users_received
0,116794,10,13,10,13
1,9458,3,2,3,2
2,117865,7,6,6,6
3,23583,9,14,7,14
4,16137,2,10,2,10


In [8]:
# Merge two dataframes to the final dataframe by "outer" join. Drop all NaN values

final_cube = df_mess.merge(df_feed, how='outer', on=['user_id']).dropna()
final_cube.head()

,user_id,messages_sent,messages_received,users_sent,users_received,likes,views,gender,age,os
0,116794,10.0,13.0,10.0,13.0,19.0,69.0,female,adult,Android
3,23583,9.0,14.0,7.0,14.0,3.0,18.0,female,adult,iOS
9,123694,5.0,0.0,5.0,0.0,1.0,23.0,male,adult,Android
21,8483,1.0,1.0,1.0,1.0,5.0,38.0,female,young,Android
27,25219,5.0,8.0,5.0,8.0,3.0,13.0,male,adult,iOS


In [9]:
# Add "event-date" column to the final dataframe

final_cube['event_date'] = datetime.date(datetime.today()) - timedelta(days=1)

In [10]:
# Form the "gender" slice by the "groupby" method. Add "dimention" column. 
# Rename column with the slice values to "dimention value" and rebuild the dataframe in the right order

df_gender = final_cube.groupby(['gender', 'event_date']).sum().reset_index()
df_gender['dimension'] = 'gender'
df_gender.rename(columns={'gender':'dimension_value'}, inplace=True)
df_gender = df_gender[['event_date', 'dimension', 'dimension_value', 'views',\
                         'likes', 'messages_received', 'messages_sent', 'users_received', 'users_sent']]
df_gender

,event_date,dimension,dimension_value,views,likes,messages_received,messages_sent,users_received,users_sent
0,2023-02-25,gender,female,4507.0,1005.0,917.0,721.0,583.0,560.0
1,2023-02-25,gender,male,5390.0,1103.0,525.0,896.0,501.0,676.0


In [11]:
# Form the "age" slice by the "pivot" method. Add "dimention" column. 
# Rename column with the slice values to "dimention value" and rebuild the dataframe in the right order

df_age = final_cube.pivot_table(values=['views', 'likes', 'messages_received', 'messages_sent',\
                                        'users_received', 'users_sent'],\
                                index=['age', 'event_date'],\
                                aggfunc='sum').reset_index()
df_age['dimension'] = 'age'
df_age.rename(columns={'age':'dimension_value'}, inplace=True)
df_age = df_age[['event_date', 'dimension', 'dimension_value', 'views',\
                         'likes', 'messages_received', 'messages_sent', 'users_received', 'users_sent']]
df_age


,event_date,dimension,dimension_value,views,likes,messages_received,messages_sent,users_received,users_sent
0,2023-02-25,age,adult,7371.0,1538.0,1179.0,1172.0,833.0,846.0
1,2023-02-25,age,old,46.0,13.0,0.0,9.0,0.0,2.0
2,2023-02-25,age,young,2480.0,557.0,263.0,436.0,251.0,388.0


In [12]:
# Form the "os" slice by the "pivot" method. Add "dimention" column. 
# Rename column with the slice values to "dimention value" and rebuild the dataframe in the right order

df_os = final_cube.pivot_table(values=['views', 'likes', 'messages_received', 'messages_sent',\
                                        'users_received', 'users_sent'],\
                                index=['event_date', 'os'],\
                                aggfunc='sum').reset_index()
df_os['dimension'] = 'os'
df_os.rename(columns={'os':'dimension_value'}, inplace=True)
df_os = df_os[['event_date', 'dimension', 'dimension_value', 'views',\
                       'likes', 'messages_received', 'messages_sent', 'users_received', 'users_sent']]
df_os

,event_date,dimension,dimension_value,views,likes,messages_received,messages_sent,users_received,users_sent
0,2023-02-25,os,Android,6399.0,1399.0,1070.0,1068.0,724.0,804.0
1,2023-02-25,os,iOS,3498.0,709.0,372.0,549.0,360.0,432.0


In [13]:
# Create the total dataframe by concatinating three slices

df_total = pd.concat([df_age, df_gender, df_os], axis=0).reset_index(drop=True)
df_total

,event_date,dimension,dimension_value,views,likes,messages_received,messages_sent,users_received,users_sent
0,2023-02-25,age,adult,7371.0,1538.0,1179.0,1172.0,833.0,846.0
1,2023-02-25,age,old,46.0,13.0,0.0,9.0,0.0,2.0
2,2023-02-25,age,young,2480.0,557.0,263.0,436.0,251.0,388.0
3,2023-02-25,gender,female,4507.0,1005.0,917.0,721.0,583.0,560.0
4,2023-02-25,gender,male,5390.0,1103.0,525.0,896.0,501.0,676.0
5,2023-02-25,os,Android,6399.0,1399.0,1070.0,1068.0,724.0,804.0
6,2023-02-25,os,iOS,3498.0,709.0,372.0,549.0,360.0,432.0
